# Job recommendation

The skills of an individual can be compared with the skills of job profiles giving an indication of how easy it would be to transition to a particular job.

This notebook will demonstrate this functionality using the users and profiles created in the setup notebook. If you have not already gone through the setup notebook please do so before running the code below.

In [1]:
import json
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000/api/v1/"

In [2]:
# Load the credentials set in the setup notebook

with open("credentials.json") as fp:
    headers = json.load(fp)

In [3]:
# Get organization details

r = requests.get(base_url + "organizations/", headers=headers)
print("Status code", r.status_code)

print("Organisation response", r.json())

Status code 200
Organisation response {'organizations': [{'id': '011509e6-e80f-487d-a5b8-8242b1078055', 'name': 'Test organization'}]}


In [4]:
# Get organization ID for the test organisation created during setup
assert 'Test organization' in [i['name'] for i in r.json()["organizations"]], "Organisation not found, run setup notebook"
organization_id = [i for i in r.json()["organizations"] if i["name"] == "Test organization"][0]["id"]

In [5]:
# Get users

r = requests.get(base_url + f"organizations/{organization_id}/users/", headers=headers)
print("Status code", r.status_code)

print("Response", r.json())
all_users = r.json()

Status code 200
Response {'users': [{'email': 'Adam@test.local', 'id': '8bd5e6ae-70cd-4afc-9a60-47961ae7a819', 'name': 'Adam', 'profile': '190e60cb-958e-478e-90b5-1b57f908e16c'}, {'email': 'Beth@test.local', 'id': 'ad0ce93b-fad6-4c01-8e5f-2c63d13152cd', 'name': 'Beth', 'profile': '0c15a884-2706-450d-a4c7-42bf9cf53531'}, {'email': 'Chris@test.local', 'id': 'ca1e6a6c-77b1-4ae6-9a06-b214cdc253e0', 'name': 'Chris', 'profile': '621d72d9-d721-4aee-90d6-66abef9bb46f'}, {'email': 'Dani@test.local', 'id': '89c5d4e8-8899-40e5-86f2-fc1f5898eec1', 'name': 'Dani', 'profile': '827a2192-c14c-4d92-b910-93e9e7f07d6a'}]}


## Getting recommendations

The job recommendation service expects 

* a list of profiles to compare against, or
* a list of profile types to compare against


In [6]:
user_id = all_users["users"][0]["id"]

data = {"profile_types":["job role"]}

r = requests.post(base_url + f"organizations/{organization_id}/users/{user_id}/job_recommendations/", 
                  json=data, headers=headers)
print("Status code", r.status_code)

results = r.json()

Status code 200


In [7]:
results

{'recommendations': [{'distance': 0.0,
   'fit': 100.0,
   'profile': {'description': '',
    'id': '95d1aa39-1e5a-4c94-9d7d-09c33ee1deac',
    'name': 'Data Engineer',
    'previous_versions': [],
    'skills': [{'level': 3.0,
      'level_name': 'Advanced',
      'skill': 'c53d24fe-7c2f-4228-bc17-2632be79c601'},
     {'level': 4.0,
      'level_name': 'Expert',
      'skill': 'da1c7012-0167-4fee-be12-23a242f03361'},
     {'level': 3.0,
      'level_name': 'Advanced',
      'skill': 'a1cac15e-2b06-433a-a257-d6bd6e7fedf3'},
     {'level': 4.0,
      'level_name': 'Expert',
      'skill': '9385de27-cd84-42a4-ab05-f8cb04f1042c'},
     {'level': 4.0,
      'level_name': 'Expert',
      'skill': 'fffc7570-09aa-4391-8bfe-c871dc0a121c'},
     {'level': 4.0,
      'level_name': 'Expert',
      'skill': '2d51e534-cb98-4a69-be40-2ec583b7d59b'},
     {'level': 4.0,
      'level_name': 'Expert',
      'skill': 'f60d2ee0-1218-4aeb-8959-dffd1399e60e'}],
    'type': 'job role',
    'user_id': None}}

In [10]:
for recommendation in results['recommendations']:
    profile = recommendation["profile"]["name"]
    fit = recommendation["fit"]
    print(f"{profile:<50}|{fit:<3}")

Data Engineer                                     |100.0
Data Analyst                                      |43.24
Software Engineer                                 |42.86
Data Scientist                                    |31.58
Software Engineer ML                              |30.0
Machine Learning Researcher                       |29.09
Machine Learning Engineer                         |28.33
Database Architects                               |27.59
Database Administrators                           |23.53
Business Intelligence Analyst                     |18.75
Applications Engineer                             |15.0
Security Engineer                                 |15.0
Full stack developer                              |14.29
Web Developers                                    |9.68
DevOps Engineer                                   |8.57
Computer and Information Systems Managers         |0.0
Systems Engineer                                  |0.0
Software Developers, Applications       

During setup we used job profiles to create the users. The user above, Adam, was given the competences from the Data Engineer profile. As the competences in the user profile and Data Engineer profile are the same it is a perfect match.